In [2]:
!pip install pandas openpyxl numpy


In [3]:
import os

folders = [
    "siddharth_trade_pipeline",
    "siddharth_trade_pipeline/src",
    "siddharth_trade_pipeline/src/parsing",
    "siddharth_trade_pipeline/data",
    "siddharth_trade_pipeline/data/raw",
    "siddharth_trade_pipeline/data/processed"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Project folders created.")


Project folders created.


In [5]:
import shutil

src = "Siddharth_Associates_sample data 2.xlsx"
dst = "siddharth_trade_pipeline/data/raw/Siddharth_Associates_sample_data_2.xlsx"

shutil.copy(src, dst)
print("File copied to:", dst)

File copied to: siddharth_trade_pipeline/data/raw/Siddharth_Associates_sample_data_2.xlsx


In [9]:
parser_code = """
import re
import pandas as pd
import numpy as np

# ----------------------------------------------------------
# MASTER REGEX PATTERNS – tuned for your dataset
# ----------------------------------------------------------

MODEL_PATTERN = re.compile(r"\\b([A-Z0-9\\-]{3,})\\b")
CAPACITY_PATTERN = re.compile(
    r"\\b(\\d+(?:\\.\\d+)?\\s*(?:ML|L|LTR|LTRS|G|GM|GRAM|KG|MM|CM|INCH))\\b",
    re.IGNORECASE
)
USD_PATTERN = re.compile(r"(\\d+(?:\\.\\d+)?)(?:\\s*)USD\\b", re.IGNORECASE)
QTY_EMBEDDED_PATTERN = re.compile(
    r"\\bPACK\\s*OF\\s*(\\d+)|PKT\\s*OF\\s*(\\d+)|BOX\\s*OF\\s*(\\d+)|\\b(\\d+)\\s*PCS\\b",
    re.IGNORECASE
)

UNIT_MAP = {
    "PCS": "PCS",
    "PC": "PCS",
    "NOS": "PCS",
    "PIECES": "PCS",
    "PIECE": "PCS",
    "KG": "KG",
    "KGS": "KG",
    "G": "G",
    "GM": "G",
    "MM": "MM"
}

def extract_model(desc: str):
    m = MODEL_PATTERN.findall(desc)
    return m[0] if m else None

def extract_capacity(desc: str):
    m = CAPACITY_PATTERN.findall(desc)
    return m[0] if m else None

def extract_usd(desc: str):
    m = USD_PATTERN.search(desc)
    return float(m.group(1)) if m else None

def extract_embedded_quantity(desc: str):
    m = QTY_EMBEDDED_PATTERN.findall(desc)
    if not m:
        return None
    flat = [x for tup in m for x in tup if x.strip() != ""]
    return int(flat[0]) if flat else None

def normalize_unit(value: str):
    if not isinstance(value, str):
        return None
    val = value.strip().upper()
    return UNIT_MAP.get(val, val)

def apply_parsing(df: pd.DataFrame, desc_col="GOODS DESCRIPTION"):
    df = df.copy()
    if desc_col not in df.columns:
        raise ValueError(f"{desc_col} not found in dataset columns.")

    desc_series = df[desc_col].fillna("").astype(str)

    df["model_number"] = desc_series.apply(extract_model)
    df["capacity_spec"] = desc_series.apply(extract_capacity)
    df["unit_price_usd_extracted"] = desc_series.apply(extract_usd)
    df["embedded_quantity"] = desc_series.apply(extract_embedded_quantity)

    unit_col_candidates = ["UNIT", "Unit of measure", "Unit of measure.1"]
    for c in unit_col_candidates:
        if c in df.columns:
            df["normalized_unit"] = df[c].apply(normalize_unit)
            break
    else:
        df["normalized_unit"] = None

    return df
"""

with open("siddharth_trade_pipeline/src/parsing/parse_goods_description.py", "w") as f:
    f.write(parser_code)

print("Parser file created.")


Parser file created.


In [10]:
import sys
from pathlib import Path
import pandas as pd

sys.path.append("siddharth_trade_pipeline/src")

from parsing.parse_goods_description import apply_parsing

RAW = Path("siddharth_trade_pipeline/data/raw/Siddharth_Associates_sample_data_2.xlsx")
OUT = Path("siddharth_trade_pipeline/data/processed/trade_parsed.csv")

df = pd.read_excel(RAW)
df_parsed = apply_parsing(df, desc_col="GOODS DESCRIPTION")

OUT.parent.mkdir(parents=True, exist_ok=True)
df_parsed.to_csv(OUT, index=False)

df_parsed.head()


,PORT CODE,DATE,IEC,HS CODE,GOODS DESCRIPTION,Master category,Model Name,Model Number,Capacity,Qty,...,UNIT PRICE_INR,TOTAL VALUE_INR,UNIT PRICE_USD,TOTAL VALUE_USD,DUTY PAID_INR,model_number,capacity_spec,unit_price_usd_extracted,embedded_quantity,normalized_unit
0,INNSA1,2025-10-28,301049751,73239990,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,NaN,NaN,NaN,NaN,NaN,...,401.725310,116500.34,4.529034,1313.419842,32736.6,TH5170,None,NaN,600.0,KG
1,INNSA1,2025-10-28,301049751,73231000,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,NaN,NaN,NaN,NaN,NaN,...,440.602218,4317901.74,4.967331,48679.839230,1898149.5,8001-2,None,NaN,2.0,KG
2,INNSA1,2025-10-23,301049751,73239990,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,NaN,NaN,NaN,NaN,NaN,...,408.893664,1637210.23,4.609850,18457.837993,460056.0,MILD,None,NaN,232.0,KG
3,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,NaN,NaN,NaN,NaN,NaN,...,378.457845,586155.51,4.219151,6534.621070,164709.7,SB-12,None,NaN,6336.0,KG
4,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,NaN,NaN,NaN,NaN,NaN,...,399.739675,4014265.76,4.456407,44752.126644,1128008.7,SB-12,None,NaN,43038.0,KG


In [12]:
df_parsed.columns = df_parsed.columns.str.lower().str.replace(" ", "_")


In [14]:
if "unit_price_usd" in df_parsed.columnsm
    bad_usd = df_parsed[df_parsed["unit_price_usd"].isna() & df_parsed["unit_price_usd"].notna()]
    print("Rows where USD missed:", len(bad_usd))
    bad_usd.head(10)


Rows where USD missed: 0


In [18]:
if "Model Number" in df_parsed.columns:
    mism = df_parsed[df_parsed["model_number"].isna() & df_parsed["Model Number"].notna()]
    print(" Rows where model missed:", len(mism))
    mism.head(10)
else:
    print("Model Number column not in dataset.")


Model Number column not in dataset.


In [19]:
df_parsed["capacity_spec"].value_counts().head(20)


capacity_spec
750ML      367
650ML       93
600ML       60
1000ML      25
900ML       21
750 ML      17
500ML       13
16CM        10
1.45L       10
800ML        5
1000 ML      4
730ML        3
900 ML       2
680ML        1
450ML        1
650 ML       1
1.5L         1
700ML        1
600 ML       1
1200ML       1
Name: count, dtype: int64

In [20]:
df_parsed["embedded_quantity"].value_counts().head(20)


embedded_quantity
3.0         104
2.0          63
6.0          52
50400.0      46
27000.0      21
25200.0      16
20400.0      14
100800.0     13
70056.0      12
16800.0      12
60000.0      11
1232.0        9
50000.0       9
22080.0       9
3648.0        9
51000.0       9
36000.0       8
59040.0       8
480.0         8
61200.0       8
Name: count, dtype: int64

In [23]:
final_csv = "siddharth_trade_pipeline/data/processed/final_trade_data.csv"
df_parsed.to_csv(final_csv, index=False)
print("Final cleaned dataset saved at:", final_csv)


Final cleaned dataset saved at: siddharth_trade_pipeline/data/processed/final_trade_data.csv


In [24]:
df_parsed.columns = df_parsed.columns.str.strip().str.upper().str.replace(" ", "_")
df_parsed.columns



Index(['PORT_CODE', 'DATE', 'IEC', 'HS_CODE', 'GOODS_DESCRIPTION',
       'MASTER_CATEGORY', 'MODEL_NAME', 'MODEL_NUMBER', 'CAPACITY', 'QTY',
       'UNIT_OF_MEASURE', 'PRICE', 'UNIT_OF_MEASURE.1', 'QUANTITY', 'UNIT',
       'UNIT_PRICE_INR', 'TOTAL_VALUE_INR', 'UNIT_PRICE_USD',
       'TOTAL_VALUE_USD', 'DUTY_PAID_INR', 'MODEL_NUMBER', 'CAPACITY_SPEC',
       'UNIT_PRICE_USD_EXTRACTED', 'EMBEDDED_QUANTITY', 'NORMALIZED_UNIT'],
      dtype='object')

In [25]:
df_parsed["GRAND_TOTAL_INR"] = (
    df_parsed["TOTAL_VALUE_INR"].fillna(0) +
    df_parsed["DUTY_PAID_INR"].fillna(0)
)
df_parsed[["TOTAL_VALUE_INR", "DUTY_PAID_INR", "GRAND_TOTAL_INR"]].head()


,TOTAL_VALUE_INR,DUTY_PAID_INR,GRAND_TOTAL_INR
0,116500.34,32736.6,149236.94
1,4317901.74,1898149.5,6216051.24
2,1637210.23,460056.0,2097266.23
3,586155.51,164709.7,750865.21
4,4014265.76,1128008.7,5142274.46


In [28]:
# --- FEATURE ENGINEERING: IMPROVED CATEGORY & SUB-CATEGORY ---

def categorize_row(desc):
    desc = str(desc).lower()

    # Category detection
    if any(x in desc for x in ["steel", "metal", "mild steel"]):
        category = "Metal / Steel"
    elif "plastic" in desc:
        category = "Plastic"
    elif "glass" in desc:
        category = "Glass"
    elif "wood" in desc:
        category = "Wooden"
    else:
        category = "Misc"

    # Sub-Category detection
    if "strainer" in desc:
        sub = "Tea Strainer"
    elif "hanger" in desc:
        sub = "Hangers"
    elif "scrubber" in desc:
        sub = "Cleaning Tools"
    elif "fruit basket" in desc:
        sub = "Baskets"
    elif "basket" in desc:
        sub = "Baskets"
    elif "cutlery" in desc:
        sub = "Cutlery Holder"
    elif "spoon" in desc or "fork" in desc:
        sub = "Cutlery / Spoons"
    elif "cloth stand" in desc:
        sub = "Cloth Stand"
    else:
        sub = "Misc"

    return pd.Series([category, sub])

df_parsed[["CATEGORY", "SUB_CATEGORY"]] = df_parsed["GOODS_DESCRIPTION"].apply(categorize_row)
df_parsed[["GOODS_DESCRIPTION", "CATEGORY", "SUB_CATEGORY"]].head(10)



,GOODS_DESCRIPTION,CATEGORY,SUB_CATEGORY
0,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,Metal / Steel,Cutlery Holder
1,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,Metal / Steel,Cleaning Tools
2,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,Metal / Steel,Cloth Stand
3,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,Metal / Steel,Tea Strainer
4,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,Metal / Steel,Tea Strainer
5,STEEL HANGER WITH HOOK (SL-89) (QTY:70056 PCS/...,Metal / Steel,Hangers
6,STEEL HANGER 10PCS SET (AM-969) (QTY: 480 SET/...,Metal / Steel,Hangers
7,AM-967 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,Metal / Steel,Baskets
8,AM-966 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,Metal / Steel,Baskets
9,AM-965 STEEL SPOON & FORK HOLDER (QTY 480 PCS/...,Metal / Steel,Cutlery / Spoons


In [29]:
import re

def extract_features(desc):
    desc = str(desc)

    # Model pattern like SB-12, AM-967, 8001-2, TH5170
    model = re.findall(r"[A-Za-z]{1,4}[-]?\d{2,4}", desc)
    model = model[0] if model else None

    # Size indicators
    if any(x in desc.lower() for x in ["big", "large"]):
        size = "Large"
    elif any(x in desc.lower() for x in ["small", "mini"]):
        size = "Small"
    else:
        size = None

    # Quantity indicators like 2PCS, 3PCS SET
    qty = re.findall(r"\d+\s*pcs", desc.lower())
    qty = qty[0].upper() if qty else None

    return pd.Series([model, size, qty])

df_parsed[["MODEL", "SIZE", "PCS_COUNT"]] = df_parsed["GOODS_DESCRIPTION"].apply(extract_features)

df_parsed.head(20)


,PORT_CODE,DATE,IEC,HS_CODE,GOODS_DESCRIPTION,MASTER_CATEGORY,MODEL_NAME,MODEL_NUMBER,CAPACITY,QTY,...,CAPACITY_SPEC,UNIT_PRICE_USD_EXTRACTED,EMBEDDED_QUANTITY,NORMALIZED_UNIT,GRAND_TOTAL_INR,CATEGORY,SUB_CATEGORY,MODEL,SIZE,PCS_COUNT
0,INNSA1,2025-10-28,301049751,73239990,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,600.0,KG,149236.94,Metal / Steel,Cutlery Holder,TH5170,None,600 PCS
1,INNSA1,2025-10-28,301049751,73231000,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,2.0,KG,6216051.24,Metal / Steel,Cleaning Tools,None,None,2PCS
2,INNSA1,2025-10-23,301049751,73239990,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,232.0,KG,2097266.23,Metal / Steel,Cloth Stand,RYX-02,None,232 PCS
3,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,6336.0,KG,750865.21,Metal / Steel,Tea Strainer,SB-12,Large,6336 PCS
4,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,43038.0,KG,5142274.46,Metal / Steel,Tea Strainer,SB-12,Large,43038 PCS
5,INNSA1,2025-10-08,301049751,73239990,STEEL HANGER WITH HOOK (SL-89) (QTY:70056 PCS/...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,70056.0,KG,8618361.19,Metal / Steel,Hangers,SL-89,None,70056 PCS
6,INNSA1,2025-10-06,301049751,73239990,STEEL HANGER 10PCS SET (AM-969) (QTY: 480 SET/...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,10.0,KG,48893.29,Metal / Steel,Hangers,AM-969,None,10PCS
7,INNSA1,2025-10-01,301049751,73239990,AM-967 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,400.0,KG,50309.80,Metal / Steel,Baskets,AM-967,None,400 PCS
8,INNSA1,2025-10-01,301049751,73239990,AM-966 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,400.0,KG,49626.59,Metal / Steel,Baskets,AM-966,None,400 PCS
9,INNSA1,2025-10-01,301049751,73239990,AM-965 STEEL SPOON & FORK HOLDER (QTY 480 PCS/...,NaN,NaN,NaN,NaN,NaN,...,None,NaN,480.0,KG,42980.34,Metal / Steel,Cutlery / Spoons,AM-965,None,480 PCS


In [30]:
def clean_pcs(x):
    if pd.isna(x):
        return None
    num = re.findall(r"\d+", str(x))
    return int(num[0]) if num else None

df_parsed["PCS_COUNT_NUM"] = df_parsed["PCS_COUNT"].apply(clean_pcs)


In [32]:
def extract_total_units(desc):
    desc = str(desc)
    match = re.search(r"QTY[:\s]*(\d+)", desc)
    return int(match.group(1)) if match else None

df_parsed["TOTAL_UNITS"] = df_parsed["GOODS_DESCRIPTION"].apply(extract_total_units)


In [33]:
def extract_unit_type(desc):
    desc = str(desc)
    match = re.search(r"QTY[:\s]*\d+\s*([A-Za-z]+)", desc)
    return match.group(1).upper() if match else None

df_parsed["UNIT_TYPE"] = df_parsed["GOODS_DESCRIPTION"].apply(extract_unit_type)


In [34]:
def extract_usd_price(desc):
    desc = str(desc)
    match = re.search(r"USD\s*([0-9]*\.?[0-9]+)", desc)
    return float(match.group(1)) if match else None

df_parsed["USD_PRICE"] = df_parsed["GOODS_DESCRIPTION"].apply(extract_usd_price)


In [35]:
df_parsed["PRICE_PER_UNIT"] = df_parsed["USD_PRICE"]


In [36]:
df_parsed["CATEGORY_CLEAN"] = df_parsed["CATEGORY"].str.replace("/", "-")
df_parsed["SUB_CATEGORY_CLEAN"] = df_parsed["SUB_CATEGORY"]


In [37]:
df_parsed.head(20)


,PORT_CODE,DATE,IEC,HS_CODE,GOODS_DESCRIPTION,MASTER_CATEGORY,MODEL_NAME,MODEL_NUMBER,CAPACITY,QTY,...,MODEL,SIZE,PCS_COUNT,PCS_COUNT_NUM,TOTAL_UNITS,UNIT_TYPE,USD_PRICE,PRICE_PER_UNIT,CATEGORY_CLEAN,SUB_CATEGORY_CLEAN
0,INNSA1,2025-10-28,301049751,73239990,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,NaN,NaN,NaN,NaN,NaN,...,TH5170,None,600 PCS,600.0,600.0,PCS,2.0300,2.0300,Metal - Steel,Cutlery Holder
1,INNSA1,2025-10-28,301049751,73231000,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,NaN,NaN,NaN,NaN,NaN,...,None,None,2PCS,2.0,336000.0,SETS,0.1390,0.1390,Metal - Steel,Cleaning Tools
2,INNSA1,2025-10-23,301049751,73239990,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,NaN,NaN,NaN,NaN,NaN,...,RYX-02,None,232 PCS,232.0,1.0,None,NaN,NaN,Metal - Steel,Cloth Stand
3,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,NaN,NaN,NaN,NaN,NaN,...,SB-12,Large,6336 PCS,6336.0,6336.0,PCS,0.9730,0.9730,Metal - Steel,Tea Strainer
4,INNSA1,2025-10-13,301049751,73239990,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,NaN,NaN,NaN,NaN,NaN,...,SB-12,Large,43038 PCS,43038.0,43038.0,PCS,0.9718,0.9718,Metal - Steel,Tea Strainer
5,INNSA1,2025-10-08,301049751,73239990,STEEL HANGER WITH HOOK (SL-89) (QTY:70056 PCS/...,NaN,NaN,NaN,NaN,NaN,...,SL-89,None,70056 PCS,70056.0,70056.0,PCS,NaN,NaN,Metal - Steel,Hangers
6,INNSA1,2025-10-06,301049751,73239990,STEEL HANGER 10PCS SET (AM-969) (QTY: 480 SET/...,NaN,NaN,NaN,NaN,NaN,...,AM-969,None,10PCS,10.0,480.0,SET,0.8190,0.8190,Metal - Steel,Hangers
7,INNSA1,2025-10-01,301049751,73239990,AM-967 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,NaN,NaN,NaN,NaN,NaN,...,AM-967,None,400 PCS,400.0,400.0,PCS,1.0310,1.0310,Metal - Steel,Baskets
8,INNSA1,2025-10-01,301049751,73239990,AM-966 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,NaN,NaN,NaN,NaN,NaN,...,AM-966,None,400 PCS,400.0,400.0,PCS,1.0170,1.0170,Metal - Steel,Baskets
9,INNSA1,2025-10-01,301049751,73239990,AM-965 STEEL SPOON & FORK HOLDER (QTY 480 PCS/...,NaN,NaN,NaN,NaN,NaN,...,AM-965,None,480 PCS,480.0,480.0,PCS,0.7340,0.7340,Metal - Steel,Cutlery / Spoons


In [38]:
material_keywords = {
    "steel": ["steel", "stainless steel", "m.s", "metal"],
    "plastic": ["plastic", "pp", "pe", "poly", "pet"],
    "glass": ["glass"],
    "wood": ["wood", "bamboo"],
    "aluminium": ["aluminium", "aluminum"],
}

def detect_material(desc):
    desc = str(desc).lower()
    for mat, keys in material_keywords.items():
        if any(k in desc for k in keys):
            return mat.title()
    return "Other"

df_parsed["MATERIAL"] = df_parsed["GOODS_DESCRIPTION"].apply(detect_material)


In [39]:
colors = ["black", "white", "silver", "gold", "red", "blue", "green", "brown"]

def detect_color(desc):
    desc = str(desc).lower()
    for c in colors:
        if c in desc:
            return c.title()
    return None

df_parsed["COLOR"] = df_parsed["GOODS_DESCRIPTION"].apply(detect_color)


In [40]:
def extract_size(desc):
    desc = str(desc)
    match = re.findall(r"(\d+\.?\d*\s*(cm|mm|inch|in|\"|l|ltr|tier))", desc, flags=re.I)
    if match:
        return ", ".join([m[0] for m in match])
    return None

df_parsed["SIZE_SPEC"] = df_parsed["GOODS_DESCRIPTION"].apply(extract_size)


In [41]:
def extract_series(model):
    if pd.isna(model):
        return None
    match = re.match(r"([A-Za-z]+)", str(model))
    return match.group(1).upper() if match else None

df_parsed["MODEL_SERIES"] = df_parsed["MODEL"].apply(extract_series)


In [42]:
def pack_type(x):
    if pd.isna(x): return None
    x = str(x).lower()
    if "set" in x: return "Set"
    if "pcs" in x or "pc" in x: return "Pieces"
    if "doz" in x or "dozen" in x: return "Dozen"
    return None

df_parsed["PACK_TYPE"] = df_parsed["UNIT_TYPE"].apply(pack_type)


In [43]:
df_parsed[[
    "GOODS_DESCRIPTION", "CATEGORY", "SUB_CATEGORY",
    "MODEL", "MODEL_SERIES", "PCS_COUNT_NUM", "TOTAL_UNITS",
    "USD_PRICE", "PRICE_PER_UNIT", "UNIT_TYPE", "PACK_TYPE",
    "MATERIAL", "COLOR", "SIZE_SPEC"
]].head(20)


,GOODS_DESCRIPTION,CATEGORY,SUB_CATEGORY,MODEL,MODEL_SERIES,PCS_COUNT_NUM,TOTAL_UNITS,USD_PRICE,PRICE_PER_UNIT,UNIT_TYPE,PACK_TYPE,MATERIAL,COLOR,SIZE_SPEC
0,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,Metal / Steel,Cutlery Holder,TH5170,TH,600.0,600.0,2.0300,2.0300,PCS,Pieces,Steel,None,None
1,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,Metal / Steel,Cleaning Tools,None,None,2.0,336000.0,0.1390,0.1390,SETS,Set,Steel,None,None
2,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,Metal / Steel,Cloth Stand,RYX-02,RYX,232.0,1.0,NaN,NaN,None,None,Steel,None,None
3,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,Metal / Steel,Tea Strainer,SB-12,SB,6336.0,6336.0,0.9730,0.9730,PCS,Pieces,Steel,None,None
4,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,Metal / Steel,Tea Strainer,SB-12,SB,43038.0,43038.0,0.9718,0.9718,PCS,Pieces,Steel,None,None
5,STEEL HANGER WITH HOOK (SL-89) (QTY:70056 PCS/...,Metal / Steel,Hangers,SL-89,SL,70056.0,70056.0,NaN,NaN,PCS,Pieces,Steel,None,None
6,STEEL HANGER 10PCS SET (AM-969) (QTY: 480 SET/...,Metal / Steel,Hangers,AM-969,AM,10.0,480.0,0.8190,0.8190,SET,Set,Steel,None,None
7,AM-967 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,Metal / Steel,Baskets,AM-967,AM,400.0,400.0,1.0310,1.0310,PCS,Pieces,Steel,None,None
8,AM-966 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,Metal / Steel,Baskets,AM-966,AM,400.0,400.0,1.0170,1.0170,PCS,Pieces,Steel,None,None
9,AM-965 STEEL SPOON & FORK HOLDER (QTY 480 PCS/...,Metal / Steel,Cutlery / Spoons,AM-965,AM,480.0,480.0,0.7340,0.7340,PCS,Pieces,Steel,None,None


In [44]:
df_parsed["clean_text"] = (
    df_parsed["GOODS_DESCRIPTION"].fillna("") + " " +
    df_parsed["MODEL"].fillna("") + " " +
    df_parsed["MATERIAL"].fillna("") + " " +
    df_parsed["SUB_CATEGORY"].fillna("")
).str.lower()


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=5000,
    ngram_range=(1,2)
)

tfidf_matrix = vectorizer.fit_transform(df_parsed["clean_text"])


In [46]:
from sklearn.cluster import KMeans

num_clusters = 8   # adjustable based on dataset diversity

kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df_parsed["PRODUCT_FAMILY"] = kmeans.fit_predict(tfidf_matrix)


In [47]:
cluster_labels = {}

for cluster_id in sorted(df_parsed["PRODUCT_FAMILY"].unique()):
    sample = df_parsed[df_parsed["PRODUCT_FAMILY"] == cluster_id]["GOODS_DESCRIPTION"].head(5).tolist()
    cluster_labels[cluster_id] = ", ".join([s[:40] for s in sample])

cluster_labels


{0: 'SS OIL POT 600ML (KWP-600B) (QTY: 39060 , SS OIL POT 900ML (KWP-1000B) (QTY: 27000, SS OIL POT 900ML (KWP-1000B) (QTY: 27000, SS OIL POT 900ML (KWP-1000B) (QTY: 26424, SS OIL POT 900ML (KWP-1000B) (QTY: 27000',
 1: 'STAINLESS STEEL SPORTS BOTTLE 750ML S-30, STAINLESS STEELSPORTS BOTTLE 650ML S115 , STAINLESS STEEL SPORTS BOTTLE 750ML SG-7, STAINLESS STEEL COLA BOTTLE 750ML SK750 , STAINLESS STEEL SPORTS BOTTLE 750ML SG-7',
 2: 'SB-12 STEEL TEA STRAINER BIG (QTY 6336 P, SB-12 STEEL TEA STRAINER BIG (QTY 43038 , NP-55 STEEL TEA STRAINER (QTY:50400 PCS/, SB-12 STEEL TEA STRAINER BIG (QTY:1026 P, SB-12 STEEL TEA STRAINER BIG (QTY:4842 P',
 3: 'MILD STEEL MULTI FUNCTION CLOTH STAND (R, MILD STEEL MULTI FUNCTION CLOTH STAND (R, MILD STEEL MULTI FUNCTION CLOTH STAND (R, MILD STEEL MULTI FUNCTION CLOTH STAND (R, MILD STEEL MULTI FUNCTION CLOTH STAND (R',
 4: 'NP-003 - STEEL ZHARA (QTY:102000 PCS/USD, NP-003 STEEL ZHARA (QTY 107400 / USD 0.4, STEEL ZHARA - NP-003 (QTY:43800 PCS/USD , NP-00

In [48]:
df_parsed["PRODUCT_FAMILY_NAME"] = df_parsed["PRODUCT_FAMILY"].map(cluster_labels)


In [49]:
def build_sku(row):
    parts = [
        str(row["MODEL_SERIES"] or "")[:4],
        str(row["PCS_COUNT_NUM"] or ""),
        str(row["MATERIAL"] or "")[:3].upper(),
        str(row["SUB_CATEGORY"] or "")[:3].upper()
    ]
    return "-".join([p for p in parts if p])

df_parsed["SKU_CODE"] = df_parsed.apply(build_sku, axis=1)


In [50]:
df_parsed[[
    "GOODS_DESCRIPTION", "MODEL", "MODEL_SERIES",
    "CATEGORY", "SUB_CATEGORY", "MATERIAL",
    "PCS_COUNT_NUM", "UNIT_TYPE", "PRODUCT_FAMILY",
    "PRODUCT_FAMILY_NAME", "SKU_CODE"
]].head(20)


,GOODS_DESCRIPTION,MODEL,MODEL_SERIES,CATEGORY,SUB_CATEGORY,MATERIAL,PCS_COUNT_NUM,UNIT_TYPE,PRODUCT_FAMILY,PRODUCT_FAMILY_NAME,SKU_CODE
0,TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS/USD 2...,TH5170,TH,Metal / Steel,Cutlery Holder,Steel,600.0,PCS,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",TH-600.0-STE-CUT
1,8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336000 SE...,None,None,Metal / Steel,Cleaning Tools,Steel,2.0,SETS,5,"8001-2 STEEL SCRUBBER 2PCS SET (QTY: 336, 8001...",2.0-STE-CLE
2,MILD STEEL MULTI FUNCTION CLOTH STAND (RYX-02-...,RYX-02,RYX,Metal / Steel,Cloth Stand,Steel,232.0,None,3,"MILD STEEL MULTI FUNCTION CLOTH STAND (R, MILD...",RYX-232.0-STE-CLO
3,SB-12 STEEL TEA STRAINER BIG (QTY 6336 PCS/USD...,SB-12,SB,Metal / Steel,Tea Strainer,Steel,6336.0,PCS,2,"SB-12 STEEL TEA STRAINER BIG (QTY 6336 P, SB-1...",SB-6336.0-STE-TEA
4,SB-12 STEEL TEA STRAINER BIG (QTY 43038 PCS/US...,SB-12,SB,Metal / Steel,Tea Strainer,Steel,43038.0,PCS,2,"SB-12 STEEL TEA STRAINER BIG (QTY 6336 P, SB-1...",SB-43038.0-STE-TEA
5,STEEL HANGER WITH HOOK (SL-89) (QTY:70056 PCS/...,SL-89,SL,Metal / Steel,Hangers,Steel,70056.0,PCS,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",SL-70056.0-STE-HAN
6,STEEL HANGER 10PCS SET (AM-969) (QTY: 480 SET/...,AM-969,AM,Metal / Steel,Hangers,Steel,10.0,SET,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",AM-10.0-STE-HAN
7,AM-967 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,AM-967,AM,Metal / Steel,Baskets,Steel,400.0,PCS,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",AM-400.0-STE-BAS
8,AM-966 STEEL FRUIT BASKET (QTY 400 PCS/USD 1.0...,AM-966,AM,Metal / Steel,Baskets,Steel,400.0,PCS,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",AM-400.0-STE-BAS
9,AM-965 STEEL SPOON & FORK HOLDER (QTY 480 PCS/...,AM-965,AM,Metal / Steel,Cutlery / Spoons,Steel,480.0,PCS,7,"TH5170 STEEL CUTLERY HOLDER (QTY:600 PCS, STEE...",AM-480.0-STE-CUT


In [55]:

# EXPORT ONLY – no SQL

import pandas as pd
from pathlib import Path

root = Path("siddharth_trade_pipeline")
processed = root / "data" / "processed"
processed.mkdir(parents=True, exist_ok=True)

# Save product dim & fact (already created earlier)
product_dim.to_csv(processed/"product_dim.csv", index=False)
shipments_fact.to_csv(processed/"shipments_fact.csv", index=False)

# Excel export
with pd.ExcelWriter(processed/"final_output.xlsx", engine="openpyxl") as writer:
    df_parsed.to_excel(writer, sheet_name="Parsed_Data", index=False)
    product_dim.to_excel(writer, sheet_name="ProductDim", index=False)
    shipments_fact.to_excel(writer, sheet_name="ShipmentsFact", index=False)

print("Export successful!")



Export successful!
